In [1]:
from Models.FakeModel import Model

import numpy as np
from checklist.editor import Editor
from checklist.perturb import Perturb
from checklist.test_types import MFT, INV, DIR
from checklist.pred_wrapper import PredictorWrapper

In [2]:
editor = Editor();

## Example formatting for FakeModel

In [3]:
model_instance = Model("abc")
res = model_instance()

print(res)

{'PE': 'Insufficient', 'KE': 'Insufficient', 'LCE': 'Insufficient', 'NF': 'Insufficient'}


## Valid Definitions

In [4]:
PE_ACCEPTABLE = [
    "energy at rest",
    "energy that a body has because of its position relative to other bodies",
    "energy stored in the car at the top of the initial drop",
]
PE_UNACCEPTABLE = [
    "energy in motion",
    "energy lost as the car goes down the hill",
    "opposite of kinetic energy",
]
PE_INSUFFICIENT = [
    "4 Joules at the top of the roller coaster",
    "measured in Joules",
    "equal to m*h*9.8",
]

KE_ACCEPTABLE = [
    "energy in motion",
    "energy that the car has because it is moving",
    "the work needed to accelerate the roller coaster car from rest",
]
KE_UNACCEPTABLE = [
    "energy at rest",
    "never lost nor gained as the car moves",
    "the opposite of potantial energy",
]
KE_INSUFFICIENT = [
    "4 Joules at the bottom of the roller coaster",
    "measured in Joules",
    "equal to .5*m*v^2",
]

LCE_ACCEPTABLE = [
    "energy cannot be created or destroyed, only transformed",
    "the total energy of an isolated system remains constant",
    "energy can be converted from one form to another, but never created or destroyed",
]
LCE_UNACCEPTABLE = [
    "energy can be created or destroyed, not transformed",
    "the total energy of an isolated system will change",
]
LCE_INSUFFICIENT = [
    "K1 + U1 = K2 + U2",
    "potential energy transforms into kinetic energy",
    "As the car goes down the hill, some energy is lost to friction as heat",
]


## Function for getting data and frequencies

In [5]:
def getData(samples, expected):
    # data = {
    #     'PE':[],
    #     'KE':[],
    #     'LCE':[],
    # }
    # frequencies = {
    #     'PE':{'Acceptable':0, 'Unacceptable':0, 'Insufficient':0, 'Not Found':0},
    #     'KE':{'Acceptable':0, 'Unacceptable':0, 'Insufficient':0, 'Not Found':0},
    #     'LCE':{'Acceptable':0, 'Unacceptable':0, 'Insufficient':0, 'Not Found':0},
    # }
    fails = 0
    for sample in samples.data:
        d = Model(sample)()
        label = d['PE'] if len(d['PE']) != 0 else d['KE'] if len(d['KE']) != 0 else d['LCE']
        score = 0
        if expected['PE'] == d['PE']:
            score += 1
        if expected['KE'] == d['KE']:
            score += 1
        if expected['LCE'] == d['LCE']:
            score += 1
            
        if score != 3:
            print("FAIL: " + sample)
            print(" Actual: " + str(d))
            print(" Expected: " + str(expected))
            fails += 1
        
        
        # data['PE'].append(d['PE'])
        # data['KE'].append(d['KE'])
        # data['LCE'].append(d['LCE'])
        # frequencies['PE'][d['PE']] += 1
        # frequencies['KE'][d['KE']] += 1
        # frequencies['LCE'][d['LCE']] += 1
    print(str(fails) + " fails out of " + str(len(samples.data)) + " tests")
    return
    # return data, frequencies

# def runtest(text):
#     res = {}
#     d = Model(text)()
#     res['PE'] = d['PE']
#     res['KE'] = d['PE']
#     res['LCE'] = d['LCE']
#     return res


# **Vocab + POS**

## Minimum Functionality Tests
### Check for correct definitions
Should return acceptable

In [6]:
samples = editor.template(
    "Potential energy is {pe_acceptable}", 
    pe_acceptable=PE_ACCEPTABLE
)
getData(samples, {"PE": "Acceptable", "KE": "Not Found", "LCE": "Not Found"})

samples = editor.template(
    "Kinetic energy is {ke_acceptable}",
    ke_acceptable=KE_ACCEPTABLE
)
getData(samples, {"PE": "Not Found", "KE": "Acceptable", "LCE": "Not Found"})

samples = editor.template(
    "The Law of Conservation of energy states {lce_acceptable}.",
    lce_acceptable=LCE_ACCEPTABLE
)
getData(samples, {"PE": "Not Found", "KE": "Not Found", "LCE": "Acceptable"})

# data, freqs = getData(samples)
# test = MFT(samples.data, labels=samples.labels, name="Test vocab + POS", capability="Vocab + POS")
# test.run()
# test.summary()

FAIL: Potential energy is energy at rest
 Actual: {'PE': 'Unacceptable', 'KE': 'Unacceptable', 'LCE': 'Not Found', 'NF': 'Insufficient'}
 Expected: {'PE': 'Acceptable', 'KE': 'Not Found', 'LCE': 'Not Found'}
FAIL: Potential energy is energy that a body has because of its position relative to other bodies
 Actual: {'PE': 'Insufficient', 'KE': 'Acceptable', 'LCE': 'Unacceptable', 'NF': 'Acceptable'}
 Expected: {'PE': 'Acceptable', 'KE': 'Not Found', 'LCE': 'Not Found'}
FAIL: Potential energy is energy stored in the car at the top of the initial drop
 Actual: {'PE': 'Insufficient', 'KE': 'Acceptable', 'LCE': 'Insufficient', 'NF': 'Acceptable'}
 Expected: {'PE': 'Acceptable', 'KE': 'Not Found', 'LCE': 'Not Found'}
3 fails out of 3 tests
FAIL: Kinetic energy is energy in motion
 Actual: {'PE': 'Insufficient', 'KE': 'Acceptable', 'LCE': 'Acceptable', 'NF': 'Unacceptable'}
 Expected: {'PE': 'Not Found', 'KE': 'Acceptable', 'LCE': 'Not Found'}
FAIL: Kinetic energy is energy that the car has be

### Check for incorrect definitions
should return unacceptable

In [7]:
samples = editor.template(
    "Potential energy is {pe_unacceptable}",
    pe_unacceptable=PE_UNACCEPTABLE
)
getData(samples, {"PE": "Unacceptable", "KE": "Not Found", "LCE": "Not Found"})

samples = editor.template(
    "Kinetic energy is {ke_unacceptable}",
    ke_unacceptable=KE_UNACCEPTABLE
)
getData(samples, {"PE": "Not Found", "KE": "Unacceptable", "LCE": "Not Found"})

samples = editor.template(
    "The Law of Conservation of energy states {lce_unacceptable}.",
    lce_unacceptable=LCE_UNACCEPTABLE
)
getData(samples, {"PE": "Not Found", "KE": "Not Found", "LCE": "Unacceptable"})

FAIL: Potential energy is energy in motion
 Actual: {'PE': 'Acceptable', 'KE': 'Not Found', 'LCE': 'Not Found', 'NF': 'Insufficient'}
 Expected: {'PE': 'Unacceptable', 'KE': 'Not Found', 'LCE': 'Not Found'}
FAIL: Potential energy is energy lost as the car goes down the hill
 Actual: {'PE': 'Acceptable', 'KE': 'Insufficient', 'LCE': 'Unacceptable', 'NF': 'Insufficient'}
 Expected: {'PE': 'Unacceptable', 'KE': 'Not Found', 'LCE': 'Not Found'}
FAIL: Potential energy is opposite of kinetic energy
 Actual: {'PE': 'Not Found', 'KE': 'Insufficient', 'LCE': 'Unacceptable', 'NF': 'Unacceptable'}
 Expected: {'PE': 'Unacceptable', 'KE': 'Not Found', 'LCE': 'Not Found'}
3 fails out of 3 tests
FAIL: Kinetic energy is energy at rest
 Actual: {'PE': 'Acceptable', 'KE': 'Unacceptable', 'LCE': 'Not Found', 'NF': 'Acceptable'}
 Expected: {'PE': 'Not Found', 'KE': 'Unacceptable', 'LCE': 'Not Found'}
FAIL: Kinetic energy is never lost nor gained as the car moves
 Actual: {'PE': 'Insufficient', 'KE': 'Not 

### Check insufficient definitions
should return insufficient

In [8]:
samples = editor.template(
    "Potential energy is {pe_insufficient}",
    pe_insufficient=PE_INSUFFICIENT
)
getData(samples, {"PE": "Insufficient", "KE": "Not Found", "LCE": "Not Found"})

samples = editor.template(
    "Kinetic energy is {ke_insufficient}",
    ke_insufficient=KE_INSUFFICIENT
)
getData(samples, {"PE": "Not Found", "KE": "Insufficient", "LCE": "Not Found"})

samples = editor.template(
    "The Law of Conservation of energy states {lce_insufficient}.",
    lce_insufficient=LCE_INSUFFICIENT
)
getData(samples, {"PE": "Not Found", "KE": "Not Found", "LCE": "Insufficient"})

FAIL: Potential energy is 4 Joules at the top of the roller coaster
 Actual: {'PE': 'Not Found', 'KE': 'Insufficient', 'LCE': 'Insufficient', 'NF': 'Acceptable'}
 Expected: {'PE': 'Insufficient', 'KE': 'Not Found', 'LCE': 'Not Found'}
FAIL: Potential energy is measured in Joules
 Actual: {'PE': 'Acceptable', 'KE': 'Acceptable', 'LCE': 'Acceptable', 'NF': 'Insufficient'}
 Expected: {'PE': 'Insufficient', 'KE': 'Not Found', 'LCE': 'Not Found'}
FAIL: Potential energy is equal to m*h*9.8
 Actual: {'PE': 'Insufficient', 'KE': 'Unacceptable', 'LCE': 'Not Found', 'NF': 'Not Found'}
 Expected: {'PE': 'Insufficient', 'KE': 'Not Found', 'LCE': 'Not Found'}
3 fails out of 3 tests
FAIL: Kinetic energy is 4 Joules at the bottom of the roller coaster
 Actual: {'PE': 'Not Found', 'KE': 'Not Found', 'LCE': 'Unacceptable', 'NF': 'Insufficient'}
 Expected: {'PE': 'Not Found', 'KE': 'Insufficient', 'LCE': 'Not Found'}
FAIL: Kinetic energy is measured in Joules
 Actual: {'PE': 'Unacceptable', 'KE': 'Not F

## Directional Tests

### Correct definitions followed by incorrect definition
correctness should not go up

In [9]:
samples = editor.template(
    "Potential energy is {pe_acceptable}. Potential energy is {pe_unacceptable}.",
    pe_acceptable=PE_ACCEPTABLE, pe_unacceptable=PE_UNACCEPTABLE
)
getData(samples, {"PE": "Acceptable", "KE": "Not Found", "LCE": "Not Found"})

samples = editor.template(
    "Kinetic energy is {ke_acceptable}. Kinetic energy is {ke_unacceptable}.",
    ke_acceptable=KE_ACCEPTABLE, ke_unacceptable=KE_UNACCEPTABLE
)
getData(samples, {"PE": "Not Found", "KE": "Acceptable", "LCE": "Not Found"})

samples = editor.template(
    "The Law of Conservation of energy states {lce_acceptable}. The Law of Conservation of energy states {lce_unacceptable}.",
    lce_acceptable=LCE_ACCEPTABLE, lce_unacceptable=LCE_UNACCEPTABLE,
)
getData(samples, {"PE": "Not Found", "KE": "Not Found", "LCE": "Acceptable"})

FAIL: Potential energy is energy at rest. Potential energy is energy in motion.
 Actual: {'PE': 'Acceptable', 'KE': 'Unacceptable', 'LCE': 'Unacceptable', 'NF': 'Not Found'}
 Expected: {'PE': 'Acceptable', 'KE': 'Not Found', 'LCE': 'Not Found'}
FAIL: Potential energy is energy at rest. Potential energy is energy lost as the car goes down the hill.
 Actual: {'PE': 'Insufficient', 'KE': 'Not Found', 'LCE': 'Insufficient', 'NF': 'Acceptable'}
 Expected: {'PE': 'Acceptable', 'KE': 'Not Found', 'LCE': 'Not Found'}
FAIL: Potential energy is energy at rest. Potential energy is opposite of kinetic energy.
 Actual: {'PE': 'Not Found', 'KE': 'Acceptable', 'LCE': 'Unacceptable', 'NF': 'Acceptable'}
 Expected: {'PE': 'Acceptable', 'KE': 'Not Found', 'LCE': 'Not Found'}
FAIL: Potential energy is energy that a body has because of its position relative to other bodies. Potential energy is energy in motion.
 Actual: {'PE': 'Insufficient', 'KE': 'Acceptable', 'LCE': 'Not Found', 'NF': 'Not Found'}
 Exp

### Incorrect Definition followed by correct definition
correctness should not go down

In [10]:
# code
samples = editor.template(
    "Potential energy is {pe_unacceptable}. Potential energy is {pe_acceptable}",
    pe_acceptable=PE_ACCEPTABLE, pe_unacceptable=PE_UNACCEPTABLE
)
getData(samples, {"PE": "Acceptable", "KE": "Not Found", "LCE": "Not Found"})

samples = editor.template(
    "Kinetic energy is {ke_unacceptable}. Kinetic energy is {ke_acceptable}",
    ke_acceptable=KE_ACCEPTABLE, ke_unacceptable=KE_UNACCEPTABLE
)
getData(samples, {"PE": "Not Found", "KE": "Acceptable", "LCE": "Not Found"})

samples = editor.template(
    "The Law of Conservation of energy states {lce_unacceptable}. The Law of Conservation of energy states {lce_acceptable}",
    lce_acceptable=LCE_ACCEPTABLE, lce_unacceptable=LCE_UNACCEPTABLE,
)
getData(samples, {"PE": "Not Found", "KE": "Not Found", "LCE": "Acceptable"})


FAIL: Potential energy is energy in motion. Potential energy is energy at rest
 Actual: {'PE': 'Unacceptable', 'KE': 'Insufficient', 'LCE': 'Acceptable', 'NF': 'Unacceptable'}
 Expected: {'PE': 'Acceptable', 'KE': 'Not Found', 'LCE': 'Not Found'}
FAIL: Potential energy is energy lost as the car goes down the hill. Potential energy is energy at rest
 Actual: {'PE': 'Insufficient', 'KE': 'Insufficient', 'LCE': 'Acceptable', 'NF': 'Not Found'}
 Expected: {'PE': 'Acceptable', 'KE': 'Not Found', 'LCE': 'Not Found'}
FAIL: Potential energy is opposite of kinetic energy. Potential energy is energy at rest
 Actual: {'PE': 'Unacceptable', 'KE': 'Acceptable', 'LCE': 'Insufficient', 'NF': 'Insufficient'}
 Expected: {'PE': 'Acceptable', 'KE': 'Not Found', 'LCE': 'Not Found'}
FAIL: Potential energy is energy in motion. Potential energy is energy that a body has because of its position relative to other bodies
 Actual: {'PE': 'Acceptable', 'KE': 'Not Found', 'LCE': 'Insufficient', 'NF': 'Insufficient

### Insufficient definition followed by correct definition
Correctness should not go down

In [11]:
# code
samples = editor.template(
    "Potential energy is {pe_insufficient}. Potential energy is {pe_acceptable}",
    pe_insufficient=PE_INSUFFICIENT, pe_acceptable=PE_ACCEPTABLE
)
getData(samples, {"PE": "Acceptable", "KE": "Not Found", "LCE": "Not Found"})

samples = editor.template(
    "Kinetic energy is {ke_insufficient}. Kinetic energy is {ke_acceptable}",
    ke_insufficient=KE_INSUFFICIENT, ke_acceptable=KE_ACCEPTABLE
)
getData(samples, {"PE": "Not Found", "KE": "Acceptable", "LCE": "Not Found"})

samples = editor.template(
    "The Law of Conservation of energy states {lce_insufficient}. The Law of Conservation of energy states {lce_acceptable}",
    lce_insufficient=LCE_INSUFFICIENT, lce_acceptable=LCE_ACCEPTABLE,
)
getData(samples, {"PE": "Not Found", "KE": "Not Found", "LCE": "Acceptable"})

FAIL: Potential energy is 4 Joules at the top of the roller coaster. Potential energy is energy at rest
 Actual: {'PE': 'Acceptable', 'KE': 'Insufficient', 'LCE': 'Unacceptable', 'NF': 'Not Found'}
 Expected: {'PE': 'Acceptable', 'KE': 'Not Found', 'LCE': 'Not Found'}
FAIL: Potential energy is measured in Joules. Potential energy is energy at rest
 Actual: {'PE': 'Insufficient', 'KE': 'Not Found', 'LCE': 'Insufficient', 'NF': 'Insufficient'}
 Expected: {'PE': 'Acceptable', 'KE': 'Not Found', 'LCE': 'Not Found'}
FAIL: Potential energy is equal to m*h*9.8. Potential energy is energy at rest
 Actual: {'PE': 'Insufficient', 'KE': 'Insufficient', 'LCE': 'Not Found', 'NF': 'Acceptable'}
 Expected: {'PE': 'Acceptable', 'KE': 'Not Found', 'LCE': 'Not Found'}
FAIL: Potential energy is measured in Joules. Potential energy is energy that a body has because of its position relative to other bodies
 Actual: {'PE': 'Insufficient', 'KE': 'Unacceptable', 'LCE': 'Acceptable', 'NF': 'Insufficient'}
 Exp

### Insufficient definition followed by incorrect definition
Should result in Acceptable

In [12]:
samples = editor.template(
    "Potential energy is {pe_insufficient}. Potential energy is {pe_unacceptable}",
    pe_insufficient=PE_INSUFFICIENT, pe_unacceptable=PE_UNACCEPTABLE
)
getData(samples, {"PE": "Unacceptable", "KE": "Not Found", "LCE": "Not Found"})

samples = editor.template(
    "Kinetic energy is {ke_insufficient}. Kinetic energy is {ke_unacceptable}",
    ke_insufficient=KE_INSUFFICIENT, ke_unacceptable=KE_UNACCEPTABLE
)
getData(samples, {"PE": "Not Found", "KE": "Unacceptable", "LCE": "Not Found"})

samples = editor.template(
    "The Law of Conservation of energy states {lce_insufficient}. The Law of Conservation of energy states {lce_unacceptable}",
    lce_insufficient=LCE_INSUFFICIENT, lce_unacceptable=LCE_UNACCEPTABLE,
)
getData(samples, {"PE": "Not Found", "KE": "Not Found", "LCE": "Unacceptable"})

FAIL: Potential energy is 4 Joules at the top of the roller coaster. Potential energy is energy in motion
 Actual: {'PE': 'Insufficient', 'KE': 'Unacceptable', 'LCE': 'Not Found', 'NF': 'Acceptable'}
 Expected: {'PE': 'Unacceptable', 'KE': 'Not Found', 'LCE': 'Not Found'}
FAIL: Potential energy is 4 Joules at the top of the roller coaster. Potential energy is energy lost as the car goes down the hill
 Actual: {'PE': 'Acceptable', 'KE': 'Acceptable', 'LCE': 'Insufficient', 'NF': 'Unacceptable'}
 Expected: {'PE': 'Unacceptable', 'KE': 'Not Found', 'LCE': 'Not Found'}
FAIL: Potential energy is 4 Joules at the top of the roller coaster. Potential energy is opposite of kinetic energy
 Actual: {'PE': 'Insufficient', 'KE': 'Unacceptable', 'LCE': 'Acceptable', 'NF': 'Unacceptable'}
 Expected: {'PE': 'Unacceptable', 'KE': 'Not Found', 'LCE': 'Not Found'}
FAIL: Potential energy is measured in Joules. Potential energy is energy in motion
 Actual: {'PE': 'Unacceptable', 'KE': 'Insufficient', 'LCE'

# Negation

## MFTs

### Negated correct definition
Should return unacceptable

In [13]:
samples = editor.template(
    "Potential energy is not {pe_acceptable}",
    pe_acceptable=PE_ACCEPTABLE
)
getData(samples, {"PE": "Unacceptable", "KE": "Not Found", "LCE": "Not Found"})

samples = editor.template(
    "Kinetic energy is not {ke_acceptable}",
    ke_acceptable=KE_ACCEPTABLE
)
getData(samples, {"PE": "Not Found", "KE": "Unacceptable", "LCE": "Not Found"})

samples = editor.template(
    "The Law of Conservation of energy does not state {lce_acceptable}.",
    lce_acceptable=LCE_ACCEPTABLE
)
getData(samples, {"PE": "Not Found", "KE": "Not Found", "LCE": "Unacceptable"})


FAIL: Potential energy is not energy at rest
 Actual: {'PE': 'Insufficient', 'KE': 'Acceptable', 'LCE': 'Acceptable', 'NF': 'Insufficient'}
 Expected: {'PE': 'Unacceptable', 'KE': 'Not Found', 'LCE': 'Not Found'}
FAIL: Potential energy is not energy that a body has because of its position relative to other bodies
 Actual: {'PE': 'Insufficient', 'KE': 'Not Found', 'LCE': 'Unacceptable', 'NF': 'Acceptable'}
 Expected: {'PE': 'Unacceptable', 'KE': 'Not Found', 'LCE': 'Not Found'}
FAIL: Potential energy is not energy stored in the car at the top of the initial drop
 Actual: {'PE': 'Acceptable', 'KE': 'Insufficient', 'LCE': 'Insufficient', 'NF': 'Unacceptable'}
 Expected: {'PE': 'Unacceptable', 'KE': 'Not Found', 'LCE': 'Not Found'}
3 fails out of 3 tests
FAIL: Kinetic energy is not energy in motion
 Actual: {'PE': 'Insufficient', 'KE': 'Insufficient', 'LCE': 'Insufficient', 'NF': 'Unacceptable'}
 Expected: {'PE': 'Not Found', 'KE': 'Unacceptable', 'LCE': 'Not Found'}
FAIL: Kinetic energy i

### Negation of incorrect definition
Should return insufficient

In [14]:
samples = editor.template(
    "Potential energy is not {pe_unacceptable}",
    pe_unacceptable=PE_UNACCEPTABLE
)
getData(samples, {"PE": "Insufficient", "KE": "Not Found", "LCE": "Not Found"})

samples = editor.template(
    "Kinetic energy is not {ke_unacceptable}",
    ke_unacceptable=KE_UNACCEPTABLE
)
getData(samples, {"PE": "Not Found", "KE": "Insufficient", "LCE": "Not Found"})

samples = editor.template(
    "The Law of Conservation of energy does not state {lce_unacceptable}.",
    lce_unacceptable=LCE_UNACCEPTABLE
)
getData(samples, {"PE": "Not Found", "KE": "Not Found", "LCE": "Insufficient"})

FAIL: Potential energy is not energy in motion
 Actual: {'PE': 'Not Found', 'KE': 'Not Found', 'LCE': 'Not Found', 'NF': 'Acceptable'}
 Expected: {'PE': 'Insufficient', 'KE': 'Not Found', 'LCE': 'Not Found'}
FAIL: Potential energy is not energy lost as the car goes down the hill
 Actual: {'PE': 'Acceptable', 'KE': 'Acceptable', 'LCE': 'Acceptable', 'NF': 'Acceptable'}
 Expected: {'PE': 'Insufficient', 'KE': 'Not Found', 'LCE': 'Not Found'}
FAIL: Potential energy is not opposite of kinetic energy
 Actual: {'PE': 'Unacceptable', 'KE': 'Unacceptable', 'LCE': 'Not Found', 'NF': 'Unacceptable'}
 Expected: {'PE': 'Insufficient', 'KE': 'Not Found', 'LCE': 'Not Found'}
3 fails out of 3 tests
FAIL: Kinetic energy is not energy at rest
 Actual: {'PE': 'Acceptable', 'KE': 'Not Found', 'LCE': 'Acceptable', 'NF': 'Insufficient'}
 Expected: {'PE': 'Not Found', 'KE': 'Insufficient', 'LCE': 'Not Found'}
FAIL: Kinetic energy is not never lost nor gained as the car moves
 Actual: {'PE': 'Insufficient', 

### Negation of insufficient
Should return insufficient

In [15]:
samples = editor.template(
    "Potential energy is not {pe_insufficient}",
    pe_insufficient=PE_INSUFFICIENT
)
getData(samples, {"PE": "Insufficient", "KE": "Not Found", "LCE": "Not Found"})

samples = editor.template(
    "Kinetic energy is not {ke_insufficient}",
    ke_insufficient=KE_INSUFFICIENT
)
getData(samples, {"PE": "Not Found", "KE": "Insufficient", "LCE": "Not Found"})

samples = editor.template(
    "The Law of Conservation of energy does not state {lce_insufficient}.",
    lce_insufficient=LCE_INSUFFICIENT
)
getData(samples, {"PE": "Not Found", "KE": "Not Found", "LCE": "Insufficient"})


FAIL: Potential energy is not 4 Joules at the top of the roller coaster
 Actual: {'PE': 'Unacceptable', 'KE': 'Not Found', 'LCE': 'Insufficient', 'NF': 'Insufficient'}
 Expected: {'PE': 'Insufficient', 'KE': 'Not Found', 'LCE': 'Not Found'}
FAIL: Potential energy is not measured in Joules
 Actual: {'PE': 'Not Found', 'KE': 'Acceptable', 'LCE': 'Insufficient', 'NF': 'Unacceptable'}
 Expected: {'PE': 'Insufficient', 'KE': 'Not Found', 'LCE': 'Not Found'}
FAIL: Potential energy is not equal to m*h*9.8
 Actual: {'PE': 'Not Found', 'KE': 'Acceptable', 'LCE': 'Not Found', 'NF': 'Not Found'}
 Expected: {'PE': 'Insufficient', 'KE': 'Not Found', 'LCE': 'Not Found'}
3 fails out of 3 tests
FAIL: Kinetic energy is not 4 Joules at the bottom of the roller coaster
 Actual: {'PE': 'Not Found', 'KE': 'Insufficient', 'LCE': 'Acceptable', 'NF': 'Not Found'}
 Expected: {'PE': 'Not Found', 'KE': 'Insufficient', 'LCE': 'Not Found'}
FAIL: Kinetic energy is not measured in Joules
 Actual: {'PE': 'Unacceptabl

# Robustness

## INV

### Misspelling of words
Should return the same value

#### Correct definitions

In [16]:
PE_TYPO_CORRECT = [
    "energy that body has becuse of its posiion relative to other bdies.",
    "energy store in te car at the tp of the initl drp.",
]
KE_TYPO_CORRECT = [
    "Kinetic enery is energy tht the car hs bcase i is movng.",
    "Kinetic ergy is the wrk neded to acelerate the rolercoaser car from rest.",
]
LCE_TYPO_CORRECT = [
    "that energy canot be create or destroy, ony transfomed.",
    "the totl energy of an isolated systm remain constnt.",
]

samples = editor.template(
    "Potential energy is {pe_typo_correct}",
    pe_typo_correct=PE_TYPO_CORRECT
)
getData(samples, {"PE": "Acceptable", "KE": "Not Found", "LCE": "Not Found"})

samples = editor.template(
    "Kinetic energy is {ke_typo_correct}",
    ke_typo_correct=KE_TYPO_CORRECT
)
getData(samples, {"PE": "Not Found", "KE": "Acceptable", "LCE": "Not Found"})

samples = editor.template(
    "The Law of Conservation of energy states {lce_typo_correct}.",
    lce_typo_correct=LCE_TYPO_CORRECT
)

getData(samples, {"PE": "Not Found", "KE": "Not Found", "LCE": "Acceptable"})

FAIL: Potential energy is energy that body has becuse of its posiion relative to other bdies.
 Actual: {'PE': 'Insufficient', 'KE': 'Unacceptable', 'LCE': 'Insufficient', 'NF': 'Insufficient'}
 Expected: {'PE': 'Acceptable', 'KE': 'Not Found', 'LCE': 'Not Found'}
FAIL: Potential energy is energy store in te car at the tp of the initl drp.
 Actual: {'PE': 'Acceptable', 'KE': 'Not Found', 'LCE': 'Insufficient', 'NF': 'Unacceptable'}
 Expected: {'PE': 'Acceptable', 'KE': 'Not Found', 'LCE': 'Not Found'}
2 fails out of 2 tests
FAIL: Kinetic energy is Kinetic enery is energy tht the car hs bcase i is movng.
 Actual: {'PE': 'Insufficient', 'KE': 'Acceptable', 'LCE': 'Acceptable', 'NF': 'Acceptable'}
 Expected: {'PE': 'Not Found', 'KE': 'Acceptable', 'LCE': 'Not Found'}
FAIL: Kinetic energy is Kinetic ergy is the wrk neded to acelerate the rolercoaser car from rest.
 Actual: {'PE': 'Not Found', 'KE': 'Insufficient', 'LCE': 'Insufficient', 'NF': 'Unacceptable'}
 Expected: {'PE': 'Not Found', '

#### Incorrect definitions

In [17]:
PE_TYPO_INCORRECT = [
    "Potential energy i energy in moton.",
    "Potentia energy is the energy lose as th car go down the hil.",
]
KE_TYPO_INCORRECT = [
    "Kinetic enery is energy tht the car hs bcase i is movng.",
    "Kinetic ergy is the wrk neded to acelerate the rolercoaser car from rest.",

]
LCE_TYPO_INCORRECT = [
    "LCE says that energy canot be create or destroy, ony transfomed.",
    "LCE state the totl energy of an isolated systm remain constnt.",
]

samples = editor.template(
    "Potential energy is {pe_typo_incorrect}",
    pe_typo_incorrect=PE_TYPO_INCORRECT
)
getData(samples, {"PE": "Unacceptable", "KE": "Not Found", "LCE": "Not Found"})

samples = editor.template(
    "Kinetic energy is {ke_typo_incorrect}",
    ke_typo_incorrect=KE_TYPO_INCORRECT
)
getData(samples, {"PE": "Not Found", "KE": "Unacceptable", "LCE": "Not Found"})

samples = editor.template(
    "The Law of Conservation of energy states {lce_typo_incorrect}.",
    lce_typo_incorrect=LCE_TYPO_INCORRECT
)
getData(samples, {"PE": "Not Found", "KE": "Not Found", "LCE": "Unacceptable"})


FAIL: Potential energy is Potential energy i energy in moton.
 Actual: {'PE': 'Unacceptable', 'KE': 'Not Found', 'LCE': 'Insufficient', 'NF': 'Insufficient'}
 Expected: {'PE': 'Unacceptable', 'KE': 'Not Found', 'LCE': 'Not Found'}
FAIL: Potential energy is Potentia energy is the energy lose as th car go down the hil.
 Actual: {'PE': 'Insufficient', 'KE': 'Insufficient', 'LCE': 'Not Found', 'NF': 'Unacceptable'}
 Expected: {'PE': 'Unacceptable', 'KE': 'Not Found', 'LCE': 'Not Found'}
2 fails out of 2 tests
FAIL: Kinetic energy is Kinetic enery is energy tht the car hs bcase i is movng.
 Actual: {'PE': 'Acceptable', 'KE': 'Insufficient', 'LCE': 'Unacceptable', 'NF': 'Unacceptable'}
 Expected: {'PE': 'Not Found', 'KE': 'Unacceptable', 'LCE': 'Not Found'}
FAIL: Kinetic energy is Kinetic ergy is the wrk neded to acelerate the rolercoaser car from rest.
 Actual: {'PE': 'Not Found', 'KE': 'Not Found', 'LCE': 'Acceptable', 'NF': 'Unacceptable'}
 Expected: {'PE': 'Not Found', 'KE': 'Unacceptabl

#### Insufficient definitions

In [18]:
PE_TYPO_INSUFFICIENT = [
    "The potentia energ at the top of te rollercoaer is 4.9 juls.",
    "Potential enrgy is measur in jules.",
]
KE_TYPO_INSUFFICIENT = [
    "The kinetic energy at the botom of the hile is 4.8 joul.",
    "Kinetic nergy transfourms into heet thrugh fricton",
]
LCE_TYPO_INSUFFICIENT = [
    "The potensial energy trainsforms in to kinetic energy becuz of the law of conservashun of energy.",
    "As the car goes down the hill, some energ is lost to fricshiun as heat.",
]

samples = editor.template(
    "Potential energy is {pe_typo_insufficient}",
    pe_typo_insufficient=PE_TYPO_INSUFFICIENT
)
getData(samples, {"PE": "Insufficient", "KE": "Not Found", "LCE": "Not Found"})

samples = editor.template(
    "Kinetic energy is {ke_typo_insufficient}",
    ke_typo_insufficient=KE_TYPO_INSUFFICIENT
)
getData(samples, {"PE": "Not Found", "KE": "Insufficient", "LCE": "Not Found"})

samples = editor.template(
    "The Law of Conservation of energy states {lce_typo_insufficient}.",
    lce_typo_insufficient=LCE_TYPO_INSUFFICIENT
)
getData(samples, {"PE": "Not Found", "KE": "Not Found", "LCE": "Insufficient"})


FAIL: Potential energy is The potentia energ at the top of te rollercoaer is 4.9 juls.
 Actual: {'PE': 'Acceptable', 'KE': 'Acceptable', 'LCE': 'Unacceptable', 'NF': 'Insufficient'}
 Expected: {'PE': 'Insufficient', 'KE': 'Not Found', 'LCE': 'Not Found'}
FAIL: Potential energy is Potential enrgy is measur in jules.
 Actual: {'PE': 'Unacceptable', 'KE': 'Insufficient', 'LCE': 'Acceptable', 'NF': 'Not Found'}
 Expected: {'PE': 'Insufficient', 'KE': 'Not Found', 'LCE': 'Not Found'}
2 fails out of 2 tests
FAIL: Kinetic energy is The kinetic energy at the botom of the hile is 4.8 joul.
 Actual: {'PE': 'Acceptable', 'KE': 'Not Found', 'LCE': 'Not Found', 'NF': 'Not Found'}
 Expected: {'PE': 'Not Found', 'KE': 'Insufficient', 'LCE': 'Not Found'}
FAIL: Kinetic energy is Kinetic nergy transfourms into heet thrugh fricton
 Actual: {'PE': 'Acceptable', 'KE': 'Acceptable', 'LCE': 'Unacceptable', 'NF': 'Unacceptable'}
 Expected: {'PE': 'Not Found', 'KE': 'Insufficient', 'LCE': 'Not Found'}
2 fails 

# Taxonomy

## INV
### Replacing words with synonyms
#### Correct definition

In [19]:
PE_SYNONYM_CORRECT = [
    "energy that body has becuse of its posiion relative to other bdies."
    "energy store in te car at the tp of the initl drp."
]
KE_SYNONYM_CORRECT = [
    "Kinetic enery is energy tht the car hs bcase i is movng."
    "Kinetic ergy is the wrk neded to acelerate the rolercoaser car from rest."
]
LCE_SYNONYM_CORRECT = [
    "that energy canot be create or destroy, ony transfomed."
    "the totl energy of an isolated systm remain constnt."
]

samples = editor.template(
    "Potential energy is {pe_synonym_correct}",
    pe_synonym_correct=PE_SYNONYM_CORRECT, nsamples=5, labels=0
)
samples += editor.template(
    "Kinetic energy is {ke_synonym_correct}",
    ke_synonym_correct=KE_SYNONYM_CORRECT, nsamples=5, labels=0
)
samples += editor.template(
    "The Law of Conservation of energy states {lce_synonym_correct}.",
    lce_synonym_correct=LCE_SYNONYM_CORRECT, nsamples=5, labels=0
)

data, freqs = getData(samples, "Acceptable")

TypeError: string indices must be integers, not 'str'

#### Incorrect definition

In [20]:
PE_SYNONYM_INCORRECT = [
    "Potential energy is the energy lost as the car rolls down the hill.",
    "Potential energy is the opposite of kinetic energy.",
]
KE_SYNONYM_INCORRECT = [
    "Kinetic energy is never lost or gained as the car moves through the rollercoaster.",
    "Kinetic energy is the opposite of potential energy.",
]
LCE_SYNONYM_INCORRECT = [
    "LCE says that energy can be made and taken.",
    "LCE states that the energy of a closed system will differ.",
    "LCE says that in an open system, energy is constant",
]

samples = editor.template(
    "Potential energy is {pe_synonym_incorrect}",
    pe_synonym_incorrect=PE_SYNONYM_INCORRECT
)
getData(samples, {"PE": "Unacceptable", "KE": "Not Found", "LCE": "Not Found"})

samples = editor.template(
    "Kinetic energy is {ke_synonym_incorrect}",
    ke_synonym_incorrect=KE_SYNONYM_INCORRECT
)
getData(samples, {"PE": "Not Found", "KE": "Unacceptable", "LCE": "Not Found"})

samples = editor.template(
    "The Law of Conservation of energy states {lce_synonym_incorrect}.",
    lce_synonym_incorrect=LCE_SYNONYM_INCORRECT
)
getData(samples, {"PE": "Not Found", "KE": "Not Found", "LCE": "Unacceptable"})

FAIL: Potential energy is Potential energy is the energy lost as the car rolls down the hill.
 Actual: {'PE': 'Unacceptable', 'KE': 'Insufficient', 'LCE': 'Unacceptable', 'NF': 'Insufficient'}
 Expected: {'PE': 'Unacceptable', 'KE': 'Not Found', 'LCE': 'Not Found'}
FAIL: Potential energy is Potential energy is the opposite of kinetic energy.
 Actual: {'PE': 'Not Found', 'KE': 'Insufficient', 'LCE': 'Unacceptable', 'NF': 'Acceptable'}
 Expected: {'PE': 'Unacceptable', 'KE': 'Not Found', 'LCE': 'Not Found'}
2 fails out of 2 tests
FAIL: Kinetic energy is Kinetic energy is never lost or gained as the car moves through the rollercoaster.
 Actual: {'PE': 'Insufficient', 'KE': 'Unacceptable', 'LCE': 'Unacceptable', 'NF': 'Unacceptable'}
 Expected: {'PE': 'Not Found', 'KE': 'Unacceptable', 'LCE': 'Not Found'}
FAIL: Kinetic energy is Kinetic energy is the opposite of potential energy.
 Actual: {'PE': 'Not Found', 'KE': 'Insufficient', 'LCE': 'Acceptable', 'NF': 'Not Found'}
 Expected: {'PE': 'N

#### Insufficient definition

In [22]:
PE_SYNONYM_INSUFFICIENT = [
    "The potential energy at the tip of the rollercoaster is 4.9 joules.",
    "Potential energy is calculated in joules.",
]
KE_SYNONYM_INSUFFICIENT = [
    "There is higher kinetic energy at the bottom of the hill than at the top.",
    "Kinetic energy turns into heat through friction",
]
LCE_SYNONYM_INSUFFICIENT = [
    "As the car goes down the hill, some energy does away to friction as heat.",
    "If there were no friction, the energy would stay the same.",
]

samples = editor.template(
    "Potential energy is {pe_synonym_insufficient}",
    pe_synonym_insufficient=PE_SYNONYM_INSUFFICIENT
)
getData(samples, {"PE": "Insufficient", "KE": "Not Found", "LCE": "Not Found"})

samples = editor.template(
    "Kinetic energy is {ke_synonym_insufficient}",
    ke_synonym_insufficient=KE_SYNONYM_INSUFFICIENT
)
getData(samples, {"PE": "Not Found", "KE": "Insufficient", "LCE": "Not Found"})

samples = editor.template(
    "The Law of Conservation of energy states {lce_synonym_insufficient}.",
    lce_synonym_insufficient=LCE_SYNONYM_INSUFFICIENT
)
getData(samples, {"PE": "Not Found", "KE": "Not Found", "LCE": "Insufficient"})

FAIL: Potential energy is The potential energy at the tip of the rollercoaster is 4.9 joules.
 Actual: {'PE': 'Acceptable', 'KE': 'Unacceptable', 'LCE': 'Not Found', 'NF': 'Insufficient'}
 Expected: {'PE': 'Insufficient', 'KE': 'Not Found', 'LCE': 'Not Found'}
FAIL: Potential energy is Potential energy is calculated in joules.
 Actual: {'PE': 'Not Found', 'KE': 'Acceptable', 'LCE': 'Insufficient', 'NF': 'Insufficient'}
 Expected: {'PE': 'Insufficient', 'KE': 'Not Found', 'LCE': 'Not Found'}
2 fails out of 2 tests
FAIL: Kinetic energy is There is higher kinetic energy at the bottom of the hill than at the top.
 Actual: {'PE': 'Unacceptable', 'KE': 'Insufficient', 'LCE': 'Not Found', 'NF': 'Acceptable'}
 Expected: {'PE': 'Not Found', 'KE': 'Insufficient', 'LCE': 'Not Found'}
FAIL: Kinetic energy is Kinetic energy turns into heat through friction
 Actual: {'PE': 'Acceptable', 'KE': 'Insufficient', 'LCE': 'Unacceptable', 'NF': 'Unacceptable'}
 Expected: {'PE': 'Not Found', 'KE': 'Insuffici

# Fairness

## INV
## Test for words being replaced in Spanish
#### Correct definitions

In [25]:
PE_SPANISH_CORRECT = [
    "Potential energy is energy that a body has because of its position relativa a other bodies.",
    "Potential energy is the stored energy of the rollercoaster car when no se esta moviendo.",
    "Potential energy is energy that has the potential to convertirse en otro form of energy.",
]
KE_SPANISH_CORRECT = [
    "Kinetic energy is the work necesario para accelerate the rollercoaster car from rest.",
    "Kinetic energy is determined by the mass del carro and the velocity with which it is moving.",
    "Kinetic energy quantifies the work an object performs debido a su movimiento",
]
LCE_SPANISH_CORRECT = [
    "LCE states that energy can be convertida from one form to another, but never created or destroyed.",
    "LCE says that if there were no friction, the potential energy en la cima del rollercoaster would be the same as the kinetic energy at the bottom of the drop.",
    "LCE is a physical law that states that energy no se puede crear or destroyed but only transformed",
]

samples = editor.template(
    "Potential energy is {pe_spanish_correct}",
    pe_spanish_correct=PE_SPANISH_CORRECT,
)
getData(samples, {"PE": "Acceptable", "KE": "Not Found", "LCE": "Not Found"})

samples = editor.template(
    "Kinetic energy is {ke_spanish_correct}",
    ke_spanish_correct=KE_SPANISH_CORRECT
)
getData(samples, {"PE": "Not Found", "KE": "Acceptable", "LCE": "Not Found"})

samples = editor.template(
    "The Law of Conservation of energy states {lce_spanish_correct}.",
    lce_spanish_correct=LCE_SPANISH_CORRECT
)
getData(samples, {"PE": "Not Found", "KE": "Not Found", "LCE": "Acceptable"})


FAIL: Potential energy is Potential energy is energy that a body has because of its position relativa a other bodies.
 Actual: {'PE': 'Unacceptable', 'KE': 'Unacceptable', 'LCE': 'Acceptable', 'NF': 'Not Found'}
 Expected: {'PE': 'Acceptable', 'KE': 'Not Found', 'LCE': 'Not Found'}
FAIL: Potential energy is Potential energy is the stored energy of the rollercoaster car when no se esta moviendo.
 Actual: {'PE': 'Acceptable', 'KE': 'Acceptable', 'LCE': 'Not Found', 'NF': 'Acceptable'}
 Expected: {'PE': 'Acceptable', 'KE': 'Not Found', 'LCE': 'Not Found'}
FAIL: Potential energy is Potential energy is energy that has the potential to convertirse en otro form of energy.
 Actual: {'PE': 'Insufficient', 'KE': 'Not Found', 'LCE': 'Acceptable', 'NF': 'Not Found'}
 Expected: {'PE': 'Acceptable', 'KE': 'Not Found', 'LCE': 'Not Found'}
3 fails out of 3 tests
FAIL: Kinetic energy is Kinetic energy is the work necesario para accelerate the rollercoaster car from rest.
 Actual: {'PE': 'Acceptable', '